# Tensor：PyTorch中最基础的计算单元

Tensor 是深度学习框架中极为基础的概念，也是 PyTroch、TensorFlow 中最重要的知识点之一，它是一种数据的存储和处理结构。
标量，也称 Scalar，是一个只有大小，没有方向的量，比如 1.8、e、10 等。
向量，也称 Vector，是一个有大小也有方向的量，比如 (1,2,3,4) 等。
矩阵，也称 Matrix，是多个向量合并在一起得到的量，比如[(1,2,3),(4,5,6)]
这种统一的数据形式，在 PyTorch 中我们称之为张量 (Tensor)。

## 直接创建

torch.tensor(data, dtype=None, device=None,requires_grad=False)

data，也就是我们要传入模型的数据。PyTorch 支持通过 list、 tuple、numpy array、scalar 等多种类型进行数据传入，并转换为 tensor
requires_grad，用于说明当前量是否需要在计算中保留对应的梯度信息。
需要注意的是，把 requires_grad 设置成 true 或者 false 要灵活处理。如果是训练过程就要设置为 true，目的是方便求导、更新参数。而到了验证或者测试过程，我们的目的是检查当前模型的泛化能力，那就要把 requires_grad 设置成 Fasle，避免这个参数根据 loss 自动更新。

## 从 NumPy 中创建

torch.from_numpy(ndarry)

## 创建特殊形式的 Tensor
- 创建零矩阵 Tensor：零矩阵顾名思义，就是所有的元素都为 0 的矩阵。
``` torch.zeros(*size, dtype=None...)```
- 创建单位矩阵 Tensor：单位矩阵是指主对角线上的元素都为 1 的矩阵
```torch.eye(size, dtype=None...)```
- 创建全一矩阵 Tensor：全一矩阵顾名思义，就是所有的元素都为 1 的矩阵。
``` torch.ones(size, dtype=None...) ```
- 创建随机矩阵 Tensor：在 PyTorch 中有几种较为经常使用的随机矩阵创建方式，分别如下

```
torch.rand(size)
torch.randn(size)
torch.normal(mean, std, size)
torch.randint(low, high, size）
```
- torch.rand 用于生成数据类型为浮点型且维度指定的随机 Tensor，随机生成的浮点数据在 0~1 区间均匀分布。
-- torch.randn 用于生成数据类型为浮点型且维度指定的随机 Tensor，随机生成的浮点数的取值满足均值为 0、方差为 1 的标准正态分布。
-- torch.normal 用于生成数据类型为浮点型且维度指定的随机 Tensor，可以指定均值和标准差。
-- torch.randint 用于生成随机整数的 Tensor，其内部填充的是在[low,high) 均匀生成的随机整数。

## Tensor 的转换

In [1]:
#Int 与 Tensor 的转换：
import torch

a = torch.tensor(1)
b = a.item()

In [2]:
#list 与 tensor 的转换：
a = [1, 2, 3]
b = torch.tensor(a)
c = b.numpy().tolist()

## Tensor 的常用操作
### 获取形状

```
>>> a=torch.zeros(2, 3, 5)
>>> a.shape
torch.Size([2, 3, 5])
>>> a.size()
torch.Size([2, 3, 5])
```
获取元素的数量

```
>>> a.numel()
30
```

### 矩阵转秩 (维度转换）
在 PyTorch 中有两个函数，分别是 permute() 和 transpose() 可以用来实现矩阵的转秩，或者说交换不同维度的数据。
> permute 函数可以对任意高维矩阵进行转置，但只有 tensor.permute() 这个调用方式

```
>>> x = torch.rand(2,3,5)
>>> x.shape
torch.Size([2, 3, 5])
>>> x = x.permute(2,1,0)
>>> x.shape
torch.Size([5, 3, 2])
```
原来的 Tensor 的形状是[2,3,5]，我们在 permute 中分别写入原来索引位置的新位置，x.permute(2,1,0)，2 表示原来第二个维度现在放在了第零个维度；同理 1 表示原来第一个维度仍旧在第一个维度；0 表示原来第 0 个维度放在了现在的第 2 个维度，形状就变成了[5,3,2]
>而另外一个函数 transpose，不同于 permute，它每次只能转换两个维度，或者说交换两个维度的数据。

```
>>> x.shape
torch.Size([2, 3, 4])
>>> x = x.transpose(1,0)
>>> x.shape
torch.Size([3, 2, 4])
```
需要注意的是，经过了 transpose 或者 permute 处理之后的数据，变得不再连续了
### 形状变换
在 PyTorch 中有两种常用的改变形状的函数，分别是 view 和 reshape。

```
>>> x = torch.randn(4, 4)
>>> x.shape
torch.Size([4, 4])
>>> x = x.view(2,8)
>>> x.shape
torch.Size([2, 8])
```

```

>>> x = x.permute(1,0)
>>> x.shape
torch.Size([8, 2])
>>> x.view(4, 4)
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.
```
结合代码可以看到，利用 permute，我们将第 0 和第 1 维度的数据进行了变换，得到了[8, 2]形状的 Tensor，在这个新 Tensor 上进行 view 操作，忽然就报错了，为什么呢？其实就是因为 view 不能处理内存不连续 Tensor 的结构。

```
>>> x = x.reshape(4, 4)
>>> x.shape
torch.Size([4, 4])
```
### 增减维度

>先来看 squeeze().如果 dim 指定的维度的值为 1，则将该维度删除，若指定的维度值不为 1，则返回原来的 Tensor。
```
>>> x = torch.rand(2,1,3)
>>> x.shape
torch.Size([2, 1, 3])
>>> y = x.squeeze(1)
>>> y.shape
torch.Size([2, 3])
>>> z = y.squeeze(1)
>>> z.shape
torch.Size([2, 3])
```
结合代码我们可以看到，我们新建了一个维度为[2, 1, 3]的 Tensor，然后将第 1 维度的数据删除，得到 y，squeeze 执行成功是因为第 1 维度的大小为 1。然而在 y 上我们打算进一步删除第 1 维度的时候，就会发现删除失败了，这是因为 y 此刻的第 1 维度的大小为 3，suqeeze 不能删除。

>unsqueeze()：这个函数主要是对数据维度进行扩充。给指定位置加上维数为 1 的维度，我们同样结合代码例子来看看

```
>>> x = torch.rand(2,1,3)
>>> y = x.unsqueeze(2)
>>> y.shape
torch.Size([2, 1, 1, 3])
```

## Tensor 的连接操作

### cat
`torch.cat(tensors, dim = 0, out = None)`
dim 的数值是多少，两个矩阵就会按照相应维度的方向链接两个 Tensor
针对维数相同

### stack
`torch.stack(inputs, dim=0)`
针对维数不同


## Tensor 的切分操作
切分的操作主要分为三种类型：chunk、split、unbind。
### chunk
chunk 的作用就是将 Tensor 按照声明的 dim，进行尽可能平均的划分
```torch.chunk(input, chunks, dim=0) ```
### split
`torch.split(tensor, split_size_or_sections, dim=0)`
split_size_or_sections 这个参数。当它为整数时，表示将 tensor 按照每块大小为这个整数的数值来切割；当这个参数为列表时，则表示将此 tensor 切成和列表中元素一样大小的块
### unbind
`torch.unbind(input, dim=0)`
unbind 是一种降维切分的方式，相当于删除一个维度之后的结果。


## Tensor 的索引操作
### index_select
`torch.index_select(tensor, dim, index)`
```py

>>> A=torch.arange(0,16).view(4,4)
>>> A
tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]])
>>> B=torch.index_select(A,0,torch.tensor([1,3]))
>>> B
tensor([[ 4,  5,  6,  7],
        [12, 13, 14, 15]])
>>> C=torch.index_select(A,1,torch.tensor([0,3]))
>>> C
tensor([[ 0,  3],
        [ 4,  7],
        [ 8, 11],
        [12, 15]])
```
### masked_select
indexed_select，它是基于给定的索引来进行数据提取的。但有的时候，我们还想通过一些判断条件来进行选择，比如提取深度学习网络中某一层中数值大于 0 的参数。
`torch.masked_select(input, mask, out=None) `

```py
>>> A=torch.rand(5)
>>> A
tensor([0.3731, 0.4826, 0.3579, 0.4215, 0.2285])
>>> C=torch.masked_select(A, A>0.3)
>>> C
tensor([0.3731, 0.4826, 0.3579, 0.4215])
```